# Build the Tkinter Interface
### Create a canvas for drawing and integrate the CNN for predictions.

In [22]:
import tkinter as tk
from PIL import Image, ImageDraw, ImageOps
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
import torchvision.transforms.functional as F

In [12]:
# CNN Model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(64 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 36)  # 36 classes (0-9, A-Z)

    def forward(self, x):
        x = nn.ReLU()(self.conv1(x))
        x = nn.MaxPool2d(2)(x)
        x = nn.ReLU()(self.conv2(x))
        x = nn.MaxPool2d(2)(x)
        x = x.view(-1, 64 * 5 * 5)
        x = nn.ReLU()(self.fc1(x))
        x = self.fc2(x)
        return x

In [13]:
# Load the trained model
model = CNN()
model.load_state_dict(torch.load("emnist_cnn.pth"))
model.eval()

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1600, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=36, bias=True)
)

In [14]:
# Label map (same as training)
labels_map = {i: chr(48 + i) if i < 10 else chr(65 + i - 10) for i in range(36)}  # 0-9, A-Z

In [34]:
class DrawingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Character Prediction")

        self.canvas = tk.Canvas(root, width=200, height=200, bg='white')
        self.canvas.pack(pady=10)

        button_frame = tk.Frame(root)
        button_frame.pack(pady=5)

        self.predict_btn = tk.Button(button_frame, text="🔍 Predict", command=self.predict,
                                     bg="#4CAF50", fg="white", font=("Arial", 12), padx=10, pady=5)
        self.predict_btn.grid(row=0, column=0, padx=5)

        self.clear_btn = tk.Button(button_frame, text="🧹 Clear", command=self.clear_canvas,
                                   bg="#f44336", fg="white", font=("Arial", 12), padx=10, pady=5)
        self.clear_btn.grid(row=0, column=1, padx=5)

        self.result_label = tk.Label(root, text="Prediction: None", font=("Arial", 16, "bold"), fg="blue")
        self.result_label.pack(pady=10)

        self.image = Image.new("L", (200, 200), 255)
        self.draw = ImageDraw.Draw(self.image)
        self.canvas.bind("<B1-Motion>", self.paint)

    def paint(self, event):
        x, y = event.x, event.y
        radius = 8
        self.canvas.create_oval(x - radius, y - radius, x + radius, y + radius, fill='black')
        self.draw.ellipse([x - radius, y - radius, x + radius, y + radius], fill=0)

    def clear_canvas(self):
        self.canvas.delete("all")
        self.image = Image.new("L", (200, 200), 255)
        self.draw = ImageDraw.Draw(self.image)
        self.result_label.config(text="Prediction: None", fg="blue")

    def predict(self):
        self.image.save("drawed-original.png")

        # Resize and invert for EMNIST
        img_resized = self.image.resize((28, 28))
        img_inverted = ImageOps.invert(img_resized)
        img_inverted.save("drawed-processed.png")

        arr = torch.tensor(np.array(img_inverted), dtype=torch.float32)
        img_tensor = arr.unsqueeze(0)  # [1, 28, 28]

        # Apply horizontal flip and 90° rotation
        img_tensor = F.rotate(F.hflip(img_tensor), 90)

        # Add batch dimension
        img_tensor = img_tensor.unsqueeze(0)  # [1, 1, 28, 28]

        with torch.no_grad():
            output = model(img_tensor)
            prediction = output.argmax(dim=1).item()
            label = labels_map[prediction]
            print(f"Predicted class: {prediction}, Label: {label}")
            self.result_label.config(text=f"🧠 Prediction: {label}", fg="green")

In [35]:
# Launch the app
root = tk.Tk()
app = DrawingApp(root)
root.mainloop()

Predicted class: 3, Label: 3
Predicted class: 7, Label: 7
Predicted class: 8, Label: 8
Predicted class: 34, Label: Y
Predicted class: 33, Label: X
Predicted class: 0, Label: 0
Predicted class: 0, Label: 0
Predicted class: 26, Label: Q
Predicted class: 3, Label: 3
Predicted class: 35, Label: Z
Predicted class: 22, Label: M
Predicted class: 26, Label: Q
Predicted class: 18, Label: I
Predicted class: 18, Label: I
Predicted class: 18, Label: I
Predicted class: 4, Label: 4
Predicted class: 15, Label: F
Predicted class: 15, Label: F
Predicted class: 12, Label: C
Predicted class: 15, Label: F
Predicted class: 29, Label: T
Predicted class: 7, Label: 7
Predicted class: 12, Label: C
Predicted class: 6, Label: 6
Predicted class: 13, Label: D
Predicted class: 10, Label: A
